In [1]:
import psycopg2
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = psycopg2.connect("dbname=football")
cur = conn.cursor()

sql = """
SELECT
match.id,
country_id,
league_id,
season,
date,
home_team_api_id,
a.team_long_name AS home_team_long_name,
away_team_api_id,
b.team_long_name AS away_team_long_name,
home_team_goal,
away_team_goal,
goal
FROM
MATCH
INNER JOIN
team a
ON match.home_team_api_id = a.team_api_id
INNER JOIN
team b
ON match.away_team_api_id = b.team_api_id
WHERE goal IS NOT NULL;
"""

cur.execute(sql)

results = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
conn.close()

In [3]:
results[0]

(7753,
 6,
 8,
 '2013/2014',
 '2014-03-29 00:00:00',
 8472,
 'Sunderland',
 8654,
 'West Ham',
 1,
 2,
 '<goal><value><event_incident_typefk>406</event_incident_typefk><elapsed>10</elapsed><comment>n</comment><sortorder>1</sortorder><type>goal</type><subtype>header</subtype><goal_type>n</goal_type><team>8654</team><player2>37169</player2><player1>47382</player1><stats><goals>1</goals><shoton>1</shoton></stats><id>3357995</id><n>202</n></value><value><event_incident_typefk>414</event_incident_typefk><elapsed>50</elapsed><comment>n</comment><sortorder>3</sortorder><type>goal</type><subtype>deflected</subtype><goal_type>n</goal_type><team>8654</team><player1>40015</player1><player2>47382</player2><stats><goals>1</goals><shoton>1</shoton></stats><id>3358213</id><n>235</n></value><value><event_incident_typefk>393</event_incident_typefk><elapsed>65</elapsed><comment>n</comment><sortorder>1</sortorder><type>goal</type><subtype>shot</subtype><goal_type>n</goal_type><team>8472</team><player1>24

In [4]:
len(results)

6711

In [5]:
colnames

['id',
 'country_id',
 'league_id',
 'season',
 'date',
 'home_team_api_id',
 'home_team_long_name',
 'away_team_api_id',
 'away_team_long_name',
 'home_team_goal',
 'away_team_goal',
 'goal']

In [6]:
df = pd.DataFrame(results, columns=colnames)
df.head()

,id,country_id,league_id,season,date,home_team_api_id,home_team_long_name,away_team_api_id,away_team_long_name,home_team_goal,away_team_goal,goal
0,7753,6,8,2013/2014,2014-03-29 00:00:00,8472,Sunderland,8654,West Ham,1,2,<goal><value><event_incident_typefk>406</event...
1,7766,6,8,2013/2014,2014-04-12 00:00:00,10194,Stoke,10261,Newcastle United,1,0,<goal><value><event_incident_typefk>407</event...
2,7765,6,8,2013/2014,2014-05-04 00:00:00,10260,Manchester United,8667,Hull,3,1,<goal><value><event_incident_typefk>411</event...
3,7754,6,8,2013/2014,2014-03-29 00:00:00,10194,Stoke,8667,Hull,1,0,<goal><value><event_incident_typefk>80</event_...
4,8472,6,8,2012/2013,2012-12-01 00:00:00,9850,Norwich,8472,Sunderland,2,1,<goal><value><event_incident_typefk>393</event...


In [7]:
df.head(1).goal.values[0]

'<goal><value><event_incident_typefk>406</event_incident_typefk><elapsed>10</elapsed><comment>n</comment><sortorder>1</sortorder><type>goal</type><subtype>header</subtype><goal_type>n</goal_type><team>8654</team><player2>37169</player2><player1>47382</player1><stats><goals>1</goals><shoton>1</shoton></stats><id>3357995</id><n>202</n></value><value><event_incident_typefk>414</event_incident_typefk><elapsed>50</elapsed><comment>n</comment><sortorder>3</sortorder><type>goal</type><subtype>deflected</subtype><goal_type>n</goal_type><team>8654</team><player1>40015</player1><player2>47382</player2><stats><goals>1</goals><shoton>1</shoton></stats><id>3358213</id><n>235</n></value><value><event_incident_typefk>393</event_incident_typefk><elapsed>65</elapsed><comment>n</comment><sortorder>1</sortorder><type>goal</type><subtype>shot</subtype><goal_type>n</goal_type><team>8472</team><player1>24159</player1><player2>25075</player2><stats><goals>1</goals><shoton>1</shoton></stats><id>3358283</id><n

In [8]:
df.head(1)[['home_team_goal', 'away_team_goal']]

,home_team_goal,away_team_goal
0,1,2


In [9]:
import xml.etree.ElementTree as ET

test = ET.fromstring(df.head(1).goal.values[0])
test

<Element 'goal' at 0x1168b1408>

In [10]:
for neigh in test.findall('value'):
    print(neigh.find('team').text)
    print(neigh.find('elapsed').text)

8654
10
8654
50
8472
65


## Let's pick 60 minutes as the time we care about

In [11]:
def score_at_minutes(goal_xml, minutes, home_team_id, away_team_id):
    parsed = ET.fromstring(goal_xml)
    home_score = 0
    away_score = 0
    for neigh in parsed.findall('value'):
        time = int(neigh.find('elapsed').text)
        if time and time < minutes:
            try:
                goal_type = neigh.find('goal_type').text
                if goal_type and goal_type in ('n', 'o', 'p'):
                    if int(neigh.find('team').text) == home_team_id:
                        home_score += 1
                    elif int(neigh.find('team').text) == away_team_id:
                        away_score += 1
                    else:
                        print("Balls")
            except:
                continue
    return home_score, away_score

score_at_minutes(df.head(1).goal.values[0], 60, 9788, 9905)

Balls
Balls


(0, 0)

In [12]:
def final_score(row):
    return row.home_team_goal, row.away_team_goal

final_score(df.head(1))

(0    1
 Name: home_team_goal, dtype: int64, 0    2
 Name: away_team_goal, dtype: int64)

In [13]:
dicty = {}
for index, row in df.iterrows():
    score_at_60 = score_at_minutes(
        row.goal,
        78,
        row.home_team_api_id,
        row.away_team_api_id
    )
    score_at_end = final_score(row)
    try:
        dicty[str(score_at_60)].append(str(score_at_end))
    except:
        dicty[str(score_at_60)] = [str(score_at_end)]

In [14]:
len(dicty)

44

In [15]:
from collections import Counter

dicty.keys()

dict_keys(['(1, 2)', '(1, 0)', '(2, 1)', '(0, 4)', '(1, 1)', '(2, 2)', '(3, 1)', '(3, 0)', '(2, 0)', '(0, 1)', '(0, 0)', '(3, 2)', '(4, 1)', '(0, 2)', '(0, 3)', '(1, 3)', '(5, 2)', '(4, 0)', '(3, 3)', '(4, 2)', '(2, 3)', '(1, 5)', '(0, 6)', '(1, 4)', '(2, 4)', '(2, 5)', '(6, 3)', '(5, 1)', '(5, 0)', '(4, 4)', '(4, 3)', '(6, 0)', '(5, 3)', '(3, 4)', '(1, 7)', '(7, 1)', '(0, 5)', '(7, 0)', '(2, 7)', '(6, 1)', '(5, 4)', '(1, 6)', '(2, 6)', '(3, 5)'])

In [16]:
new_dicty = dict((key, Counter(val)) for key, val in dicty.items())

In [17]:
new_dicty

{'(1, 2)': Counter({'(1, 2)': 196,
          '(1, 3)': 66,
          '(2, 2)': 68,
          '(1, 4)': 8,
          '(3, 3)': 2,
          '(3, 2)': 14,
          '(2, 1)': 12,
          '(2, 3)': 10,
          '(2, 4)': 1,
          '(0, 3)': 3,
          '(3, 1)': 5,
          '(4, 2)': 1,
          '(5, 1)': 1}),
 '(1, 0)': Counter({'(1, 0)': 516,
          '(0, 1)': 14,
          '(2, 0)': 170,
          '(1, 1)': 146,
          '(1, 2)': 24,
          '(2, 1)': 35,
          '(3, 0)': 24,
          '(4, 0)': 9,
          '(2, 2)': 4,
          '(2, 3)': 1,
          '(4, 1)': 1,
          '(0, 2)': 2,
          '(-1, -1)': 1,
          '(3, 1)': 2,
          '(1, 3)': 1}),
 '(2, 1)': Counter({'(3, 1)': 90,
          '(2, 1)': 276,
          '(5, 1)': 4,
          '(2, 2)': 73,
          '(4, 1)': 17,
          '(3, 2)': 24,
          '(4, 2)': 5,
          '(1, 2)': 22,
          '(5, 0)': 2,
          '(1, 3)': 2,
          '(2, 3)': 10,
          '(3, 0)': 11,
          '(3, 3)'

In [18]:
[(i, new_dicty['(1, 1)'][i] / len(new_dicty['(1, 1)']) * 100.0) for i in new_dicty['(1, 1)']]

[('(1, 2)', 869.2307692307692),
 ('(2, 1)', 1146.1538461538462),
 ('(1, 1)', 3623.0769230769233),
 ('(1, 3)', 100.0),
 ('(0, 3)', 38.46153846153847),
 ('(2, 2)', 146.15384615384613),
 ('(3, 2)', 46.15384615384615),
 ('(2, 0)', 146.15384615384613),
 ('(3, 1)', 207.6923076923077),
 ('(1, 4)', 7.6923076923076925),
 ('(0, 2)', 61.53846153846154),
 ('(3, 0)', 92.3076923076923),
 ('(4, 1)', 7.6923076923076925)]

In [19]:
sum(new_dicty['(1, 1)'].values())

844

In [20]:
for key in new_dicty['(1, 1)']:
    print(key)

(1, 2)
(2, 1)
(1, 1)
(1, 3)
(0, 3)
(2, 2)
(3, 2)
(2, 0)
(3, 1)
(1, 4)
(0, 2)
(3, 0)
(4, 1)


In [21]:
new_dicty['(1, 1)']['(1, 4)']

1

In [22]:
import operator
tah = new_dicty['(0, 2)']
sorted([(e, tah[e]/sum(tah.values())) for e in tah], key=lambda x: x[1], reverse=True)

[('(0, 2)', 0.4859550561797753),
 ('(1, 2)', 0.1853932584269663),
 ('(0, 3)', 0.15730337078651685),
 ('(1, 1)', 0.05056179775280899),
 ('(1, 3)', 0.033707865168539325),
 ('(2, 2)', 0.033707865168539325),
 ('(0, 4)', 0.019662921348314606),
 ('(2, 1)', 0.011235955056179775),
 ('(2, 3)', 0.0056179775280898875),
 ('(1, 4)', 0.0056179775280898875),
 ('(2, 0)', 0.0028089887640449437),
 ('(0, 5)', 0.0028089887640449437),
 ('(1, 5)', 0.0028089887640449437),
 ('(3, 3)', 0.0028089887640449437)]

In [23]:
small_df = df[df.season != '2018/2019']
small_dicty = {}
for index, row in small_df.iterrows():
    score_at_60 = score_at_minutes(
        row.goal,
        78,
        row.home_team_api_id,
        row.away_team_api_id
    )
    score_at_end = final_score(row)
    try:
        small_dicty[str(score_at_60)].append(str(score_at_end))
    except:
        small_dicty[str(score_at_60)] = [str(score_at_end)]
        
small_new_dicty = dict((key, Counter(val)) for key, val in small_dicty.items())

In [24]:
import operator
tah = small_new_dicty['(2, 0)']
sorted([(e, tah[e]/sum(tah.values())) for e in tah], key=lambda x: x[1], reverse=True)

[('(2, 0)', 0.55078125),
 ('(3, 0)', 0.177734375),
 ('(2, 1)', 0.134765625),
 ('(3, 1)', 0.037109375),
 ('(1, 1)', 0.03125),
 ('(4, 0)', 0.02734375),
 ('(2, 2)', 0.01953125),
 ('(3, 2)', 0.005859375),
 ('(4, 1)', 0.00390625),
 ('(5, 0)', 0.00390625),
 ('(3, 3)', 0.001953125),
 ('(2, 3)', 0.001953125),
 ('(1, 2)', 0.001953125),
 ('(0, 3)', 0.001953125)]

In [25]:
new_df = df[df.season == '2018/2019']
newer_dicty = {}
for index, row in new_df.iterrows():
    score_at_60 = score_at_minutes(
        row.goal,
        78,
        row.home_team_api_id,
        row.away_team_api_id
    )
    score_at_end = final_score(row)
    try:
        newer_dicty[str(score_at_60)].append(str(score_at_end))
    except:
        newer_dicty[str(score_at_60)] = [str(score_at_end)]
        
new_new_dicty = dict((key, Counter(val)) for key, val in newer_dicty.items())

In [26]:
new_new_dicty

{'(3, 1)': Counter({'(3, 1)': 9, '(4, 1)': 2}),
 '(0, 0)': Counter({'(0, 0)': 37,
          '(0, 1)': 8,
          '(1, 1)': 1,
          '(0, 3)': 1,
          '(2, 0)': 2,
          '(1, 0)': 6,
          '(-1, -1)': 3,
          '(2, 1)': 1,
          '(0, 2)': 1}),
 '(1, 2)': Counter({'(1, 2)': 12,
          '(2, 2)': 9,
          '(3, 1)': 3,
          '(1, 3)': 1,
          '(3, 2)': 2,
          '(2, 3)': 2}),
 '(1, 0)': Counter({'(1, 1)': 13,
          '(2, 0)': 10,
          '(1, 0)': 37,
          '(2, 1)': 4,
          '(-1, -1)': 1,
          '(2, 2)': 1,
          '(3, 0)': 2,
          '(1, 2)': 1}),
 '(0, 1)': Counter({'(1, 1)': 11,
          '(1, 2)': 2,
          '(0, 1)': 31,
          '(0, 2)': 8,
          '(0, 4)': 1,
          '(1, 0)': 1,
          '(2, 2)': 1,
          '(0, 3)': 1,
          '(2, 0)': 1}),
 '(2, 1)': Counter({'(3, 1)': 6,
          '(2, 1)': 12,
          '(2, 3)': 1,
          '(1, 2)': 4,
          '(4, 1)': 3,
          '(2, 2)': 6,
        

In [27]:
small_new_dicty['(0, 0)']

Counter({'(0, 0)': 460,
         '(2, 0)': 9,
         '(0, 1)': 105,
         '(1, 0)': 124,
         '(1, 1)': 24,
         '(0, 2)': 18,
         '(1, 2)': 4,
         '(2, 1)': 8,
         '(2, 2)': 2,
         '(3, 0)': 3})

In [28]:
new_new_dicty['(0, 0)']

Counter({'(0, 0)': 37,
         '(0, 1)': 8,
         '(1, 1)': 1,
         '(0, 3)': 1,
         '(2, 0)': 2,
         '(1, 0)': 6,
         '(-1, -1)': 3,
         '(2, 1)': 1,
         '(0, 2)': 1})

In [29]:
tah = small_new_dicty['(3, 0)']
sorted([(e, tah[e]/sum(tah.values())) for e in tah], key=lambda x: x[1], reverse=True)

[('(3, 0)', 0.5205992509363296),
 ('(4, 0)', 0.20599250936329588),
 ('(3, 1)', 0.1348314606741573),
 ('(4, 1)', 0.04119850187265917),
 ('(2, 1)', 0.0299625468164794),
 ('(5, 0)', 0.026217228464419477),
 ('(2, 2)', 0.00749063670411985),
 ('(5, 1)', 0.00749063670411985),
 ('(4, 2)', 0.00749063670411985),
 ('(3, 2)', 0.00749063670411985),
 ('(2, 3)', 0.003745318352059925),
 ('(6, 1)', 0.003745318352059925),
 ('(6, 0)', 0.003745318352059925)]

In [30]:
mah = new_new_dicty['(1, 0)']
sorted([(e, mah[e]/sum(mah.values())) for e in mah], key=lambda x: x[1], reverse=True)

[('(1, 0)', 0.5362318840579711),
 ('(1, 1)', 0.18840579710144928),
 ('(2, 0)', 0.14492753623188406),
 ('(2, 1)', 0.057971014492753624),
 ('(3, 0)', 0.028985507246376812),
 ('(-1, -1)', 0.014492753623188406),
 ('(2, 2)', 0.014492753623188406),
 ('(1, 2)', 0.014492753623188406)]

In [31]:
sum([mah[e] for e in mah if int(e.split(',')[0].lstrip('()')) + int(e.split(',')[1].rstrip(')').strip()) >= 3]) / sum(mah.values())

0.11594202898550725

In [32]:
sum([tah[e] for e in tah if int(e.split(',')[0].lstrip('()')) + int(e.split(',')[1].rstrip(')').strip()) >= 3]) / sum(tah.values())

1.0

In [33]:
df.head()

,id,country_id,league_id,season,date,home_team_api_id,home_team_long_name,away_team_api_id,away_team_long_name,home_team_goal,away_team_goal,goal
0,7753,6,8,2013/2014,2014-03-29 00:00:00,8472,Sunderland,8654,West Ham,1,2,<goal><value><event_incident_typefk>406</event...
1,7766,6,8,2013/2014,2014-04-12 00:00:00,10194,Stoke,10261,Newcastle United,1,0,<goal><value><event_incident_typefk>407</event...
2,7765,6,8,2013/2014,2014-05-04 00:00:00,10260,Manchester United,8667,Hull,3,1,<goal><value><event_incident_typefk>411</event...
3,7754,6,8,2013/2014,2014-03-29 00:00:00,10194,Stoke,8667,Hull,1,0,<goal><value><event_incident_typefk>80</event_...
4,8472,6,8,2012/2013,2012-12-01 00:00:00,9850,Norwich,8472,Sunderland,2,1,<goal><value><event_incident_typefk>393</event...


## ELO

In [41]:
HISTORY = 10
k_factor = 32
elo_width = 400.

def calculate_new_elos(rating_a, rating_b, score_a, k_factor, elo_width):
    """Calculates and returns the new Elo ratings for two players.
    score_a is 1 for a win by player A, 0 for a loss by player A, or 0.5 for a draw.
    """

    e_a = expected_result(rating_a, rating_b, elo_width)
    e_b = 1. - e_a
    new_rating_a = rating_a + k_factor * (score_a - e_a)
    score_b = 1. - score_a
    new_rating_b = rating_b + k_factor * (score_b - e_b)
    return new_rating_a, new_rating_b
    

def expected_result(elo_a, elo_b, elo_width):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a


def update_end_of_season(elos):
    """Regression towards the mean
    
    Following 538 nfl methods
    https://fivethirtyeight.com/datalab/nfl-elo-ratings-are-back/
    """
    diff_from_mean = elos - np.mean(elos)
    elos -= diff_from_mean/3
    return elos

In [35]:
df.columns

Index(['id', 'country_id', 'league_id', 'season', 'date', 'home_team_api_id',
       'home_team_long_name', 'away_team_api_id', 'away_team_long_name',
       'home_team_goal', 'away_team_goal', 'goal'],
      dtype='object')

In [36]:
def build_season_teams(frame, season, previous_elos=None):
    season_teams = pd.DataFrame(pd.unique(frame[frame.season == season][['home_team_long_name', 'away_team_long_name']].values.ravel('K')), columns=['home_team_long_name'])
#    season_teams = frame[frame.season == season].home_team_long_name.drop_duplicates().reset_index()
    season_teams.loc[:, 'elo'] = 1000
    if previous_elos is not None:
        meany = np.mean(previous_elos.elo)
        previous_elos.loc[:, 'updated_elo'] = previous_elos.elo.apply(lambda x: x - (x - meany)/3)
        joiny = season_teams.merge(previous_elos, how='left', left_on='home_team_long_name', right_on='team_long_name')
        season_teams = joiny[['home_team_long_name', 'updated_elo']]
        season_teams.columns = ['home_team_long_name', 'elo']
        season_teams.fillna(1000., inplace=True)
    season_teams = season_teams[['home_team_long_name', 'elo']]
    season_teams.columns = ['team_long_name', 'elo']
    return season_teams


def calculate_result(row):
    if row.home_team_goal > row.away_team_goal:
        return 1
    elif row.home_team_goal == row.away_team_goal:
        return 0.5
    else:
        return 0

df.loc[:, 'home_elo'] = 1000.
df.loc[:, 'away_elo'] = 1000.
for key, frame in df.groupby(['country_id', 'league_id']):
    country, league = key
    sorted_frame = frame.sort_values(by='date')
    earliest_season = sorted_frame.head(1).season.values[0]
    season_frame = build_season_teams(sorted_frame, earliest_season)
    for index, row in sorted_frame.iterrows():
        new_season = row.season
        if new_season != earliest_season:
            earliest_season = new_season
            season_frame = build_season_teams(sorted_frame, new_season, season_frame)
            print("New season")
        
        home = row.home_team_long_name
        away = row.away_team_long_name
        home_elo = season_frame[season_frame.team_long_name == home].elo.values[0]
        away_elo = season_frame[season_frame.team_long_name == away].elo.values[0]
        df.loc[df.id == row.id, 'home_elo'] = home_elo
        df.loc[df.id == row.id, 'away_elo'] = away_elo
        expected_results = expected_result(home_elo, away_elo, elo_width)
        result = calculate_result(row)
        new_home_elo, new_away_elo = calculate_new_elos(home_elo, away_elo, result, k_factor, elo_width)
        season_frame.loc[season_frame.team_long_name == home, 'elo'] = new_home_elo
        season_frame.loc[season_frame.team_long_name == away, 'elo'] = new_away_elo
            
        

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season


In [37]:
df.head()

,id,country_id,league_id,season,date,home_team_api_id,home_team_long_name,away_team_api_id,away_team_long_name,home_team_goal,away_team_goal,goal,home_elo,away_elo
0,7753,6,8,2013/2014,2014-03-29 00:00:00,8472,Sunderland,8654,West Ham,1,2,<goal><value><event_incident_typefk>406</event...,981.965594,972.570717
1,7766,6,8,2013/2014,2014-04-12 00:00:00,10194,Stoke,10261,Newcastle United,1,0,<goal><value><event_incident_typefk>407</event...,977.639499,970.314256
2,7765,6,8,2013/2014,2014-05-04 00:00:00,10260,Manchester United,8667,Hull,3,1,<goal><value><event_incident_typefk>411</event...,1076.975539,971.063257
3,7754,6,8,2013/2014,2014-03-29 00:00:00,10194,Stoke,8667,Hull,1,0,<goal><value><event_incident_typefk>80</event_...,961.061327,973.621643
4,8472,6,8,2012/2013,2012-12-01 00:00:00,9850,Norwich,8472,Sunderland,2,1,<goal><value><event_incident_typefk>393</event...,1015.063435,985.278237


In [38]:
df.sort_values(by='home_elo').tail()

,id,country_id,league_id,season,date,home_team_api_id,home_team_long_name,away_team_api_id,away_team_long_name,home_team_goal,away_team_goal,goal,home_elo,away_elo
2613,9730,7,9,2017/2018,2018-04-29 00:00:00,9847,Paris Saint-Germain,9747,Guingamp,2,2,<goal><value><event_incident_typefk>39</event_...,1233.759736,952.676943
5597,16433,10,15,2015/2016,2016-04-16 00:00:00,9885,Juventus,8540,Palermo,4,0,<goal><value><event_incident_typefk>411</event...,1235.951515,883.940895
2333,13163,8,12,2014/2015,2015-05-23 00:00:00,8634,Barcelona,9783,Deportivo La Coruna,2,2,<goal><value><event_incident_typefk>406</event...,1239.647373,986.909490
1809,8567,6,8,2017/2018,2018-03-04 00:00:00,8456,Manchester City,8455,Chelsea,1,0,<goal><value><event_incident_typefk>393</event...,1245.105838,1091.685850
72,8632,6,8,2017/2018,2018-05-09 00:00:00,8456,Manchester City,10204,Brighton,3,1,<goal><value><event_incident_typefk>393</event...,1259.126906,962.006253


In [39]:
np.mean(df.home_team_goal)

1.5398599314558188

In [40]:
np.mean(df.away_team_goal)

1.1701683802711966

In [74]:
HOME_TEAM_BASE = np.mean(df.home_team_goal)
AWAY_TEAM_BASE = np.mean(df.away_team_goal)

k_factor = 32
score_k_factor = 0.05

def calculate_new_elos_score(rating_a, rating_b, score_a, home, k_factor):
    """Calculates and returns the new Elo ratings for two players.
    score_a is 1 for a win by player A, 0 for a loss by player A, or 0.5 for a draw.
    """
    if home:
        e_a = expected_result_score(rating_a, rating_b, HOME_TEAM_BASE)
    else:
        e_a = expected_result_score(rating_a, rating_b, AWAY_TEAM_BASE)
    new_rating_a = rating_a + (k_factor * (score_a - e_a))
    new_rating_b = rating_b + (k_factor * (e_a - score_a))
    return new_rating_a, new_rating_b
    

def expected_result_score(elo_a, elo_b, base):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expect_a = (elo_a/base) * (elo_b/base) * base
    return expect_a


def update_end_of_season(elos):
    """Regression towards the mean
    
    Following 538 nfl methods
    https://fivethirtyeight.com/datalab/nfl-elo-ratings-are-back/
    """
    diff_from_mean = elos - np.mean(elos)
    elos -= diff_from_mean/3
    return elos

In [75]:
def build_season_frame(frame, season, base_score, previous_elos=None):
    season_teams = pd.DataFrame(pd.unique(frame[frame.season == season][['home_team_long_name', 'away_team_long_name']].values.ravel('K')), columns=['home_team_long_name'])
    season_teams.loc[:, 'elo'] = base_score
    if previous_elos is not None:
        meany = np.mean(previous_elos.elo)
        previous_elos.loc[:, 'updated_elo'] = previous_elos.elo.apply(lambda x: x - (x - meany)/3)
        joiny = season_teams.merge(previous_elos, how='left', left_on='home_team_long_name', right_on='team_long_name')
        season_teams = joiny[['home_team_long_name', 'updated_elo']]
        season_teams.columns = ['home_team_long_name', 'elo']
        season_teams.fillna(base_score, inplace=True)
    season_teams = season_teams[['home_team_long_name', 'elo']]
    season_teams.columns = ['team_long_name', 'elo']
    return season_teams

def get_elos(frame, names):
    return [frame[frame.team_long_name == entry].elo.values[0] for entry in names]

df.loc[:, 'home_elo'] = 1000.
df.loc[:, 'away_elo'] = 1000.
df.loc[:, 'home_attack_elo'] = HOME_TEAM_BASE
df.loc[:, 'home_defence_elo'] = AWAY_TEAM_BASE
df.loc[:, 'away_attack_elo'] = AWAY_TEAM_BASE
df.loc[:, 'away_defence_elo'] = HOME_TEAM_BASE

for key, frame in df.groupby(['country_id', 'league_id']):
    country, league = key
    sorted_frame = frame.sort_values(by='date')
    earliest_season = sorted_frame.head(1).season.values[0]
    season_frame = build_season_frame(sorted_frame, earliest_season, 1000.)
    home_attack_frame = build_season_frame(sorted_frame, earliest_season, HOME_TEAM_BASE)
    away_attack_frame = build_season_frame(sorted_frame, earliest_season, AWAY_TEAM_BASE)
    home_defence_frame = build_season_frame(sorted_frame, earliest_season, AWAY_TEAM_BASE)
    away_defence_frame = build_season_frame(sorted_frame, earliest_season, HOME_TEAM_BASE)    
    for index, row in sorted_frame.iterrows():
        new_season = row.season
        if new_season != earliest_season:
            earliest_season = new_season
            season_frame = build_season_frame(sorted_frame, new_season, 1000., season_frame)
            home_attack_frame = build_season_frame(sorted_frame, new_season, HOME_TEAM_BASE, home_attack_frame)
            away_attack_frame = build_season_frame(sorted_frame, new_season, AWAY_TEAM_BASE, away_attack_frame)
            home_defence_frame = build_season_frame(sorted_frame, new_season, AWAY_TEAM_BASE, home_defence_frame)
            away_defence_frame = build_season_frame(sorted_frame, new_season, HOME_TEAM_BASE, away_defence_frame)                
            print("New season")
        
        home = row.home_team_long_name
        away = row.away_team_long_name
        home_elo, away_elo = get_elos(season_frame, (home, away))
        home_attack_elo = get_elos(home_attack_frame, (home,))[0]
        home_defence_elo = get_elos(home_defence_frame, (home,))[0]
        away_attack_elo = get_elos(away_attack_frame, (away,))[0]
        away_defence_elo = get_elos(away_defence_frame, (away,))[0]
        df.loc[df.id == row.id, 'home_elo'] = home_elo
        df.loc[df.id == row.id, 'away_elo'] = away_elo
        df.loc[df.id == row.id, 'home_attack_elo'] = home_attack_elo
        df.loc[df.id == row.id, 'away_attack_elo'] = away_attack_elo
        df.loc[df.id == row.id, 'home_defence_elo'] = home_defence_elo
        df.loc[df.id == row.id, 'away_defence_elo'] = away_defence_elo        
        expected_results = expected_result(home_elo, away_elo, elo_width)
        result = calculate_result(row)
        new_home_elo, new_away_elo = calculate_new_elos(home_elo, away_elo, result, k_factor, elo_width)
        season_frame.loc[season_frame.team_long_name == home, 'elo'] = new_home_elo
        season_frame.loc[season_frame.team_long_name == away, 'elo'] = new_away_elo
        
        home_team_goals = row.home_team_goal
        away_team_goals = row.away_team_goal
        if home_team_goals >= 0 and away_team_goals >= 0:
            new_home_attack_elo, new_away_defence_elo = calculate_new_elos_score(home_attack_elo, away_defence_elo, home_team_goals, True, score_k_factor)
            new_away_attack_elo, new_home_defence_elo = calculate_new_elos_score(away_attack_elo, home_defence_elo, away_team_goals, False, score_k_factor)        
            home_attack_frame.loc[home_attack_frame.team_long_name == home, 'elo'] = new_home_attack_elo
            home_defence_frame.loc[home_defence_frame.team_long_name == home, 'elo'] = new_home_defence_elo
            away_attack_frame.loc[away_attack_frame.team_long_name == away, 'elo'] = new_away_attack_elo
            away_defence_frame.loc[away_defence_frame.team_long_name == away, 'elo'] = new_away_defence_elo
        


New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season
New season


In [48]:
home

'Wigan'

In [79]:
df.sort_values(by='away_attack_elo').tail()

,id,country_id,league_id,season,date,home_team_api_id,home_team_long_name,away_team_api_id,away_team_long_name,home_team_goal,away_team_goal,goal,home_elo,away_elo,home_attack_elo,home_defence_elo,away_attack_elo,away_defence_elo
1003,12994,8,12,2013/2014,2014-05-11 00:00:00,10268,Elche,8634,Barcelona,0,0,<goal />,986.531861,1219.210201,1.283977,1.397135,1.987517,2.015811
2783,13629,8,12,2017/2018,2018-04-15 00:00:00,9864,Malaga,8633,Real Madrid,1,2,<goal><value><event_incident_typefk>130</event...,958.331329,1151.473392,1.221288,1.841394,1.987844,2.691767
4859,13810,8,12,2017/2018,2018-02-03 00:00:00,8558,Espanyol,8634,Barcelona,1,1,<goal><value><event_incident_typefk>406</event...,1032.303306,1202.686491,1.045890,1.158506,1.995339,3.568150
151,7882,6,8,2013/2014,2014-05-03 00:00:00,9826,Crystal Palace,8650,Liverpool,3,3,<goal><value><event_incident_typefk>406</event...,1010.637765,1110.398595,1.325585,0.999890,2.031528,1.782979
4733,13753,8,12,2017/2018,2018-05-13 00:00:00,8581,Levante,8634,Barcelona,5,4,<goal><value><event_incident_typefk>721</event...,1030.937006,1215.386139,1.349976,1.287625,2.098235,3.787309


In [50]:
home_attack_elo = get_elos(home_attack_frame, (home))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [53]:
home_attack_frame[home_attack_frame.team_long_name == 'Wigan'].elo.values[0]

1.5398599314558188

In [56]:
nice = ('Wigan',)

In [57]:
[home_attack_frame[home_attack_frame.team_long_name == entry].elo.values[0] for entry in nice]

[1.5398599314558188]